In [1]:
import pandas as pd
import numpy as np

import ha_tools

In [2]:
query = """
select ar.action
     , sr.oid
     , sr.submitted_date dt
     , sr.match_type_id
     , tc.description match_type
     , rl.session_id
     , nvl(lead.accepts,0) accepts
     , nvl(lead.closed_leads,0) closed_leads
     , nvl(lead.won_leads,0) won_leads
     , nvl(lead.call_tracking_leads,0) call_tracking_leads
     , nvl(lead.called_leads,0) called_leads
     , nvl(lead.connections,0) connections
     , nvl(lead.leads_w_credit_req,0) leads_w_credit_req
     , nvl(lead.leads_w_appr_cr,0) leads_w_appr_cr
     , (nvl(atr.grev,0) - nvl(cr.cr_amount,0)) net_rev
     , nvl(atr.grev,0) gross_rev
  from wisen_data.sm_action_rule ar
     , wisen_data.sm_action_rule_log rl
     , wisen_data.bv_ws_service_requests sr
     , wisen_data.sm_sr_match_type_cd tc
     , (select atr.sr_id sr_oid
             , sum(atr.amount) grev
          from wisen_data.sm_account_trans atr
             , wisen_data.sm_trans_type tt
          where atr.trans_type = tt.trans_type
            and atr.trans_sub_type = tt.trans_sub_type
            and atr.trans_date >= '29-JUN-2018'
            and atr.trans_date < sysdate
            and atr.sr_id is not null
            and tt.gl_id = 4100000 --Gross Lead Revenue
          group by atr.sr_id
       ) atr
     , (select atc.sr_id sr_oid
             , sum(ta.amount) cr_amount
          from wisen_data.sm_account_trans atc
             , wisen_data.sm_trans_apply ta
             , wisen_data.sm_account_trans atd
             , wisen_data.sm_trans_type tt
         where atc.trans_type = tt.trans_type
           and atc.trans_sub_type = tt.trans_sub_type
           and atc.trans_id = ta.credit_trans_id
           and ta.debit_trans_id = atd.trans_id
           and atc.trans_sub_type <> 'HSNMLP'
           and ta.status > 0
           and atd.trans_date >= '29-JUN-2018'
           and atd.trans_date < sysdate
           and atc.sr_id is not null
           and tt.gl_id = 1250000 --Lead Credits
         group by atc.sr_id
        ) cr
     , (select l.sr_oid
             , count(distinct l.lead_id) accepts
             , count(distinct case when l.lead_mgmt_status_id = 901 or l.lead_mgmt_status_id = 902 then l.lead_id end) closed_leads
             , count(distinct case when l.lead_mgmt_status_id = 901 then l.lead_id end) won_leads
             , count(distinct l.lead_ids_w_call_tracking) call_tracking_leads
             , count(distinct case when l.call_tracking_id is not null then l.lead_id end) called_leads
             , count(distinct case when l.call_duration > 60 then l.lead_id end) connections
             , count(distinct case when l.leads_w_credit > 0 then l.lead_id end) leads_w_credit_req
             , count(distinct case when l.leads_w_appr_credit > 0 then l.lead_id end) leads_w_appr_cr
          from (select l.lead_id
                     , l.sr_oid
                     , l.lead_type_id
                     , l.create_datetime lead_create_dt
                     , lce.lead_id lead_ids_w_call_tracking
                     , min(ct.call_tracking_id)keep(dense_rank first order by ct.create_datetime) call_tracking_id
                     , min(ct.call_duration)keep(dense_rank first order by ct.create_datetime) call_duration
                     , min(ct.create_datetime)keep(dense_rank first order by ct.create_datetime) call_dt
                     , l.lead_mgmt_status_id
                     , (case when ctr.ctr_cnt is not null then 1 else 0 end) leads_w_credit
                     , (case when ctr.appr_ctr_cnt > 0 then 1 else 0 end) leads_w_appr_credit
                  from wisen_data.sm_lead l
                     , wisen_data.sm_lead_call_ext_map lce             --multiple records (distinct controls for leads_called count)
                     , wisen_data.sm_sp_call_tracking ct 
                     , wisen_data.sm_sp_provisioned_phone_number pn
                     , (select /* INDEX (c SM_CTR_CDT_IDX)*/
                               c.sr_oid
                             , c.sp_id
                             , count(c.ctr_id) ctr_cnt
                             , count(case when c.ctr_status_id = 2 then c.ctr_id end) appr_ctr_cnt
                             , count(case when c.ctr_status_id = 3 then c.ctr_id end) den_ctr_cnt
                          from wisen_data.sm_ctr c
                         where c.create_datetime >= '29-JUN-2018'
                           and c.create_datetime <  sysdate
                         group by c.sr_oid
                             , c.sp_id)  ctr
                 where l.lead_id = lce.lead_id(+)
                   and lce.lead_call_ext_map_id = ct.lead_call_ext_map_id(+)
                   and l.sp_id = pn.sp_id(+)
                   and l.sr_oid = ctr.sr_oid (+)
                   and l.sp_id = ctr.sp_id (+)
                   and l.lead_status = 2
                   and l.test_lead = 'N'
                   and ct.call_direction_id(+) in (2,3)
                   and pn.expiration_datetime(+) >= l.create_datetime
                   and pn.create_datetime(+) < l.create_datetime
                   and l.create_datetime >= '29-JUN-2018'
                   and l.create_datetime < sysdate
                 group by l.lead_id
                        , l.sr_oid
                        , l.lead_type_id
                        , l.create_datetime
                        , lce.lead_id
                        , l.lead_mgmt_status_id
                        , (case when ctr.ctr_cnt is not null then 1 else 0 end)
                        , (case when ctr.appr_ctr_cnt > 0 then 1 else 0 end)) l
         group by l.sr_oid)  lead
 where ar.rule_id = rl.rule_id
   and rl.session_id = sr.session_id
   and sr.match_type_id = tc.match_type_id
   and sr.oid = atr.sr_oid (+)
   and sr.oid = cr.sr_oid (+)
   and sr.oid = lead.sr_oid (+) 
   and ar.action_key = 'SpCallingNotif_Test_3_2018_SITE-27016'
   and rl.create_datetime >= '29-JUN-2018'
   and rl.create_datetime <  sysdate
   and sr.submitted_date >= '29-JUN-2018'
   and sr.submitted_date <  sysdate
   and sr.test_sr = 'N'
   and sr.request_status not in ('TEST','PENDING')
   and sr.user_id != 0
   and rownum <= 1000000
"""

In [3]:
with ha_tools.db_connection('ksade', 'pipeline2018') as conn:
    df = pd.read_sql(query, conn)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4456 entries, 0 to 4455
Data columns (total 16 columns):
ACTION                 4456 non-null object
OID                    4456 non-null int64
DT                     4456 non-null datetime64[ns]
MATCH_TYPE_ID          4456 non-null int64
MATCH_TYPE             4456 non-null object
SESSION_ID             4456 non-null object
ACCEPTS                4456 non-null int64
CLOSED_LEADS           4456 non-null int64
WON_LEADS              4456 non-null int64
CALL_TRACKING_LEADS    4456 non-null int64
CALLED_LEADS           4456 non-null int64
CONNECTIONS            4456 non-null int64
LEADS_W_CREDIT_REQ     4456 non-null int64
LEADS_W_APPR_CR        4456 non-null int64
NET_REV                4456 non-null float64
GROSS_REV              4456 non-null float64
dtypes: datetime64[ns](1), float64(2), int64(10), object(3)
memory usage: 557.1+ KB


In [5]:
df['DT2'] = df['DT'].dt.floor('d')

In [6]:
df['DT2'].head()

0   2018-06-29
1   2018-06-29
2   2018-06-29
3   2018-06-29
4   2018-06-29
Name: DT2, dtype: datetime64[ns]

In [7]:
grouped = df.groupby(['DT2','ACTION'], as_index=False)

In [8]:
df2 = grouped.agg({'OID': 'count', 'SESSION_ID': 'count', 'ACCEPTS': 'sum','CLOSED_LEADS': 'sum','WON_LEADS':'sum',
             'CALL_TRACKING_LEADS':'sum','CALLED_LEADS':'sum','CONNECTIONS':'sum','LEADS_W_CREDIT_REQ':'sum',
             'LEADS_W_APPR_CR':'sum', 'NET_REV':'sum','GROSS_REV':'sum'}).rename(columns={'OID': 'SR_COUNT',
                                                                                          'SESSION_ID': 'SESSION_COUNT'})

In [9]:
df2.head()

,DT2,ACTION,CALLED_LEADS,CLOSED_LEADS,ACCEPTS,SESSION_COUNT,GROSS_REV,LEADS_W_CREDIT_REQ,CALL_TRACKING_LEADS,LEADS_W_APPR_CR,SR_COUNT,WON_LEADS,CONNECTIONS,NET_REV
0,2018-06-29,SpCallingNotif_Test_3_2018_SITE-27016_Ctrl,142,35,216,133,6152.85,11,177,6,133,19,80,5987.42
1,2018-06-29,SpCallingNotif_Test_3_2018_SITE-27016_Test,157,38,241,141,7206.05,24,196,13,141,11,71,6681.70
2,2018-06-30,SpCallingNotif_Test_3_2018_SITE-27016_Ctrl,144,41,221,116,5505.01,22,179,11,116,17,68,5177.93
3,2018-06-30,SpCallingNotif_Test_3_2018_SITE-27016_Test,125,31,184,102,5624.77,22,155,7,102,12,57,5377.64
4,2018-07-01,SpCallingNotif_Test_3_2018_SITE-27016_Ctrl,102,28,154,81,4654.60,17,128,14,81,11,47,4111.03


In [10]:
df2['ACCEPTS/SR'] = df2['ACCEPTS']/df2['SR_COUNT']
df2['SRS/SESSION'] = df2['SR_COUNT']/df2['SESSION_COUNT']
df2['WIN_RATE'] = df2['WON_LEADS']/df2['CLOSED_LEADS']
df2['CALL_RATE'] = df2['CALLED_LEADS']/df2['CALL_TRACKING_LEADS']
df2['CONNECTION_RATE'] = df2['CONNECTIONS']/df2['CALL_TRACKING_LEADS']
df2['CONTACT_ON_CALL_RATE'] = df2['CONNECTIONS']/df2['CALLED_LEADS']
df2['CREDIT_REQUEST_RATE'] = df2['LEADS_W_CREDIT_REQ']/df2['ACCEPTS']
df2['CREDIT_APPROVAL_RATE'] = df2['LEADS_W_APPR_CR']/df2['LEADS_W_CREDIT_REQ']
df2['NREV/SR'] = df2['NET_REV']/df2['SR_COUNT']
df2['GREV/SR'] = df2['GROSS_REV']/df2['SR_COUNT']
df2['DATE'] = df2['DT2']

In [11]:
df2.loc[df2['ACTION'] == 'SpCallingNotif_Test_3_2018_SITE-27016_Ctrl', 'ACTION_KEY'] = 'Control'
df2.loc[df2['ACTION'] == 'SpCallingNotif_Test_3_2018_SITE-27016_Test', 'ACTION_KEY'] = 'Test'

In [12]:
del df2['DT2']
df2.head()

,ACTION,CALLED_LEADS,CLOSED_LEADS,ACCEPTS,SESSION_COUNT,GROSS_REV,LEADS_W_CREDIT_REQ,CALL_TRACKING_LEADS,LEADS_W_APPR_CR,SR_COUNT,...,WIN_RATE,CALL_RATE,CONNECTION_RATE,CONTACT_ON_CALL_RATE,CREDIT_REQUEST_RATE,CREDIT_APPROVAL_RATE,NREV/SR,GREV/SR,DATE,ACTION_KEY
0,SpCallingNotif_Test_3_2018_SITE-27016_Ctrl,142,35,216,133,6152.85,11,177,6,133,...,0.542857,0.802260,0.451977,0.563380,0.050926,0.545455,45.018195,46.262030,2018-06-29,Control
1,SpCallingNotif_Test_3_2018_SITE-27016_Test,157,38,241,141,7206.05,24,196,13,141,...,0.289474,0.801020,0.362245,0.452229,0.099585,0.541667,47.387943,51.106738,2018-06-29,Test
2,SpCallingNotif_Test_3_2018_SITE-27016_Ctrl,144,41,221,116,5505.01,22,179,11,116,...,0.414634,0.804469,0.379888,0.472222,0.099548,0.500000,44.637328,47.456983,2018-06-30,Control
3,SpCallingNotif_Test_3_2018_SITE-27016_Test,125,31,184,102,5624.77,22,155,7,102,...,0.387097,0.806452,0.367742,0.456000,0.119565,0.318182,52.721961,55.144804,2018-06-30,Test
4,SpCallingNotif_Test_3_2018_SITE-27016_Ctrl,102,28,154,81,4654.60,17,128,14,81,...,0.392857,0.796875,0.367188,0.460784,0.110390,0.823529,50.753457,57.464198,2018-07-01,Control


In [13]:
out_csv = 'B:\Projects\SALT, ED\Automate AB Testing\Automate_AB_Testing.csv'

In [14]:
df2.to_csv(out_csv)

In [15]:
#df3 = df2[['DATE','ACTION','ACCEPTS/SR','SRS/SESSION','WIN_RATE','CALL_RATE','CONNECTION_RATE','CONTACT_ON_CALL_RATE',
#          'CREDIT_APPROVAL_RATE','NREV/SR','GREV/SR']]

In [16]:
#df3.head()

In [ ]:
def bootstrapped_ci(column, ci = 90):
    sample_means = []

    for i in range(1000):
        sample = column.sample(len(column), replace = True)
        sample_means.append(sample.mean())
    
    alpha = 100 - ci
    
    average = column.mean()
    median = column.median()
    
    lb = np.percentile(sample_means, alpha / 2)
    ub = np.percentile(sample_means, 100 - (alpha / 2))
    
    return lb - average, average - ub, average